In [9]:
import pandas as pd
import glob
import os

In [4]:
# Use glob to find all files matching the pattern
dataset_filenames = glob.glob('../data/*_keyword_frequency.csv')
dataset_filenames

['../data/authors_keyword_frequency.csv',
 '../data/index_keyword_frequency.csv',
 '../data/all_keyword_frequency.csv']

In [8]:
# Load the stop words dataset
stop_words_df = pd.read_csv('../data/StopWord.csv')
stop_words = stop_words_df['StopWord'].tolist()
print(f"Number of stop words: {len(stop_words)}")

Number of stop words: 1478


In [14]:
# Initialize an empty list to store cleaned DataFrames
cleaned_dataframes = []

# Process each dataset
for filename in dataset_filenames:
    # Load the dataset
    df = pd.read_csv(filename)
    
    # Filter out rows where the keyword is in the stop words list
    cleaned_df = df[~df['Concepts'].isin(stop_words)]
    
    # Keep only the 'Concepts' and 'Total' columns
    cleaned_df = cleaned_df[['Concepts', 'Total']]
    
    # Append the cleaned DataFrame to the list
    cleaned_dataframes.append(cleaned_df)

    # Save the cleaned DataFrame to a new CSV file in the 'dataset' folder
    cleaned_filename = os.path.join('../data', os.path.basename(filename).replace('keyword_frequency', 'cleaned_keyword_frequency'))
    cleaned_df.to_csv(cleaned_filename, index=False)